In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [2]:
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.MONAI_Dict_Dataset_Module import MONAI_CSV_Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_TILE import UNET_TILE_lit
from monai.visualize import matshow3d
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

2023-05-02 12:40:05,948 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpuyb8khr2
2023-05-02 12:40:05,949 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpuyb8khr2/_remote_module_non_scriptable.py
cpu


In [3]:
KAGGLE_DIR = PATH / "kaggle"

INPUT_DIR = KAGGLE_DIR / "input"

COMPETITION_DATA_DIR = INPUT_DIR / "vesuvius-challenge-ink-detection"

TRAIN_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_train_0.5.csv"
TEST_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_test_1.0.csv"

In [4]:
dataset = MONAI_CSV_Scrolls_Dataset(
                                z_dim=4,
                                batch_size=1,
                                data_csv_path=TRAIN_DATA_CSV_PATH,
                                num_workers=8,
                                num_samples=16,
                                patch_size=(256,256),
                                val_fragment_id=1,
                                on_gpu=False,

                            )


In [5]:
lit_model = UNET_TILE_lit(
        use_wandb = False,
        z_dim = 16,
        patch_size = (256,256),
        sw_batch_size=8 ,
        eta_min = 1e-8,
        t_max = 250,
        max_epochs = 1000,
        weight_decay =  0.001,
        learning_rate = 0.0001,
        gamma = 0.85,)

#lit_model = lit_model.load_from_checkpoint('logs/FocalDICE_512_monai_unet_16_cont/lightning_logs/version_0/checkpoints/epoch=171-step=344.ckpt', 
#                                          #patch_size = (512,512),
#                                          learning_rate = 0.00001,
#                                           weight_decay =  .01,
#                                           eta_min = 1e-9,
#                                         sw_batch_size = 8)

In [6]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="FBETA",
    mode="max",
    dirpath="logs/Full_16_768to1024_batch/",
    filename="FocalDice_768{epoch:02d}{FBETA:.2f}{recall:.2f}{precision:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='cpu',
        #benchmark=True,
        max_epochs=100,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/Full_16_768to1024_batch/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/Full_16_768to1024_batch/',
        #overfit_batches=1,
        #precision=16,
        accumulate_grad_batches=2, 
        callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/FocalDice_768epoch=123FBETA=0.30recall=0.77precision=0.26.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Full_16_768to1024_batch/FocalDice_768epoch=151FBETA=0.41recall=0.56precision=0.39.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'

2023-05-02 12:40:06,734 - GPU available: True (mps), used: False
2023-05-02 12:40:06,735 - TPU available: False, using: 0 TPU cores
2023-05-02 12:40:06,735 - IPU available: False, using: 0 IPUs
2023-05-02 12:40:06,735 - HPU available: False, using: 0 HPUs
# train: 2
# val: 1
Adjusting learning rate of group 0 to 1.0000e-04.
2023-05-02 12:40:06,849 - 
   | Name                 | Type                  | Params
----------------------------------------------------------------
0  | metrics              | ModuleDict            | 0     
1  | model                | UNet                  | 6.5 M 
2  | loss                 | MaskedLoss            | 0     
3  | mine_focal           | FocalLoss             | 0     
4  | weighted_bce_loss    | BCEWithLogitsLoss     | 0     
5  | loss_dice            | DiceLoss              | 0     
6  | loss_tversky         | TverskyLoss           | 0     
7  | loss_bce             | SoftBCEWithLogitsLoss | 0     
8  | loss_focal           | FocalLoss             |

/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/gregory/PROJECT_ML/VESUVIUS_Challenge/logs/Full_16_768to1024_batch exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


RuntimeError: Error while merging hparams: the keys ['z_dim'] are present in both the LightningModule's and LightningDataModule's hparams but have different values.